In [1]:
# import system modules
import sys
import os
os.system('for a in /sys/bus/pci/devices/*; do echo 0 | tee -a $a/numa_node>/dev/null; done') # get rid of NUMA node warnings in my docker: https://github.com/tensorflow/tensorflow/issues/42738
import gc

# import standard numerical modules
import numpy as np
import math

# import machine learning modules
import tensorflow as tf
import keras.backend as K

gpu = tf.config.list_physical_devices('GPU') # make sure GPU usage is enabled
tf.config.experimental.set_virtual_device_configuration(gpu[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=int(7.5*1024))]) # make sure tensorflow doesn't use too much memorty and crashes
print(gpu) 

import DCTR


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# load data

In [2]:
# directory with pre converted lhe files as numpy arrays
data_dir = '../Data' # modify as needed


In [4]:
# Load POWHEG hvq x0 datasets
# x0_nrm for training, x0_plt and x0_plt_nrm for calculating stats used to decide which model performs best
# only contain tt-pair; every event has order: 
    # tt-pair, top, anti-top
# every particle has arguments: 
    # [pt, y, phi, mass, eta, E, PID, w, theta]
    # [0 , 1, 2  , 3   , 4  , 5, 6  , 7, 8    ]

# POWHEG hvq
x0_nrm = []
x0_nrm = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/01-02_normed_converted_lhe.npz', i=3)[:9553938] # 9553938 num of NNLO samples
print(f'POWHEG hvq x0_nrm.shape:     {x0_nrm.shape}')

# plotting data; different from training data; for calculating stats
x0_plt = []
x0_plt = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/03-04_converted_lhe.npz', i=3)[:9553938]
print(f'POWHEG hvq x0_plt.shape:     {x0_plt.shape}')

x0_plt_nrm = []
x0_plt_nrm = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/03-04_normed_converted_lhe.npz', i=3)[:9553938]
print(f'POWHEG hvq x0_plt_nrm.shape: {x0_plt_nrm.shape}')



POWHEG hvq x0_nrm.shape:     (9553938, 3, 9)
POWHEG hvq x0_plt.shape:     (9553938, 3, 9)
POWHEG hvq x0_plt_nrm.shape: (9553938, 3, 9)


In [5]:
# MiNNLO x1
# training data
x1_nrm = []
x1_nrm = DCTR.load_dataset(f'{data_dir}/MiNNLO/converted_with_13TeV_NLO/normed_converted_lhe.npz', i=3)
print(f'MiNNLO all particles x1_nrm.shape: {x1_nrm.shape}')

# plotting data
x1_plt = []
x1_plt = DCTR.load_dataset(f'{data_dir}/MiNNLO/converted_with_13TeV_NLO/converted_lhe.npz', i=3)
print(f'MiNNLO all particles x1_plt.shape: {x1_plt.shape}')


MiNNLO all particles x1_nrm.shape: (9553938, 3, 9)
MiNNLO all particles x1_plt.shape: (9553938, 3, 9)


In [6]:
# get normalized event generator weights | all weigths = +/-1
x0_wgt = x0_nrm[:, 0, 7].copy()

x0_plt_wgt = x0_plt_nrm[:, 0, 7].copy() 

x1_wgt = x1_nrm[:, 0, 7].copy()


In [7]:
# check sum of wgt for both classes
print(f'x0 wgt sum: {np.sum(x0_wgt)}')
print(f'x1 wgt sum: {np.sum(x1_wgt)}')


x0 wgt sum: 9476758.0
x1 wgt sum: 7692656.0


In [8]:
# delete eta (pseudorapidity) and Energy -> Train only with [pt, y, phi, m, PID]

# delete energy
x0_nrm = np.delete(x0_nrm, 5, -1)
x0_plt_nrm = np.delete(x0_plt_nrm, 5, -1)
x1_nrm = np.delete(x1_nrm, 5, -1)

# delete eta
x0_nrm = np.delete(x0_nrm, 4, -1)
x0_plt_nrm = np.delete(x0_plt_nrm, 4, -1)
x1_nrm = np.delete(x1_nrm, 4, -1)


In [9]:
# prep arrays for training
x_train, x_val, y_train, y_val, wgt_train, wgt_val = DCTR.prep_arrays(x0_nrm, x1_nrm, val=0.25)

# bring into shape for training loop
train_data = (x_train, y_train, x_val, y_val, wgt_train, wgt_val)
plt_data = (x0_plt , x0_plt_nrm, x1_plt, x1_wgt)


2024-05-04 09:26:32.214812: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 09:26:32.980649: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7680 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5


In [10]:
%whos


Variable     Type           Data/Info
-------------------------------------
DCTR         module         <module 'DCTR' from '/tf/<...>CTR_FP/20240503/DCTR.py'>
K            module         <module 'keras.backend' f<...>ckages/keras/backend.py'>
data_dir     str            ../Data
gc           module         <module 'gc' (built-in)>
gpu          list           n=1
math         module         <module 'math' (built-in)>
np           module         <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
os           module         <module 'os' from '/usr/lib/python3.8/os.py'>
plt_data     tuple          n=4
sys          module         <module 'sys' (built-in)>
tf           module         <module 'tensorflow' from<...>/tensorflow/__init__.py'>
train_data   tuple          n=6
wgt_train    ndarray        14330907: 14330907 elems, type `float64`, 114647256 bytes (109.33614349365234 Mb)
wgt_val      ndarray        4776969: 4776969 elems, type `float64`, 38215752 bytes (36.44538116455078 Mb)
x0_n

In [11]:
K.clear_session() 

# clear temp arrays and variables from memory
del wgt_train, wgt_val, x0_nrm, x0_plt, x0_plt_nrm, x0_plt_wgt, x0_wgt
del x1_nrm, x1_wgt, x1_plt, x_train, x_val, y_train, y_val

print(gc.collect()) # cpu gabage collection to free up memory from discarded temp arrays


1175


In [15]:
%whos


Variable     Type      Data/Info
--------------------------------
DCTR         module    <module 'DCTR' from '/tf/<...>CTR_FP/20240503/DCTR.py'>
K            module    <module 'keras.backend' f<...>ckages/keras/backend.py'>
data_dir     str       ../Data
gc           module    <module 'gc' (built-in)>
gpu          list      n=1
math         module    <module 'math' (built-in)>
np           module    <module 'numpy' from '/us<...>kages/numpy/__init__.py'>
os           module    <module 'os' from '/usr/lib/python3.8/os.py'>
plt_data     tuple     n=4
sys          module    <module 'sys' (built-in)>
tf           module    <module 'tensorflow' from<...>/tensorflow/__init__.py'>
train_data   tuple     n=6


# training loop

In [16]:
train_dir = './train_20240504' # where to save models during training


In [17]:
# start training loop
''' train_loop() necessary arguments
train_data, plt_data

default arguments:
model=None, lowest_chi2 = 1e6, train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train',
batch_sizes=[4*8192, 8*8192, 16*8192, 32*8192], repeat=5, super_epochs=35, super_patience = 5, epochs = 8, starting_super_epoch = 1, 
input_dim=5, Phi_sizes = (100,100,128), F_sizes = (128,100,100), loss = 'mse', dropout=0.0, l2_reg=0.0, 
Phi_acts=('linear', 'gelu', 'gelu'), F_acts=('gelu', 'gelu', 'linear'), output_act='sigmoid', learning_rate=0.001

returns: best_model_list, lowest_chi2_list, lowest_loss_list
'''
best_model_list, lowest_chi2_list, lowest_loss_list = DCTR.train_loop(train_data, plt_data, batch_sizes=[24*8192, 32*8192], repeat=7, super_epochs=5, 
                                                                      train_dir = train_dir, epochs=10, learning_rate=0.001)


best_model = best_model_list[-1]
lowest_chi2 = lowest_chi2_list[-1]


starting super_epoch 1

starting training with batch_size: 196608 and 10 epochs
starting with weights from model: None
starting run 0 of super_epoch 1 with batch_size 196608


2024-05-04 09:32:50.271198: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 7.50G (8053063680 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-05-04 09:32:50.396897: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7680 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5


reset neural network weights


2024-05-04 09:33:00.985391: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tenso

...

INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tenso

2024-05-04 09:56:34.163105: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7680 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5


INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tenso

2024-05-04 09:58:09.871700: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7680 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5


INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tenso

In [18]:
K.clear_session() 
print(gc.collect()) # cpu gabage collection

# continue training loop with best model as starting point and smaller batch_ sizes and less repeats

model_list, chi2_list, loss_list = DCTR.train_loop(train_data, plt_data, model=best_model, lowest_chi2=lowest_chi2, batch_sizes=[16*8192, 24*8192], repeat=4, super_epochs=4, 
                                                   starting_super_epoch=6, train_dir = train_dir, epochs=10, learning_rate=0.001)

if len(model_list) >= 1:
    best_model_list.extend(model_list)
    lowest_chi2_list.extend(chi2_list)

best_model = best_model_list[-1]
lowest_chi2 = lowest_chi2_list[-1]


81651
starting super_epoch 6

starting training with batch_size: 131072 and 10 epochs
starting with weights from model: ./train_20240504/super_epoch_3/run_3/s-3_b-262144_r-3.tf
starting run 0 of super_epoch 6 with batch_size 131072
loaded neural network model: ./train_20240504/super_epoch_3/run_3/s-3_b-262144_r-3.tf
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00012960000021848827.
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets

Epoch 9: ReduceLROnPlateau reducing learning rate to 7.775999838486313e-05.
INFO:tensorflow:Assets written t

In [19]:
K.clear_session() 
print(gc.collect()) # cpu gabage collection

# continue training loop with best model as starting point and smaller batch_sizes, less repeats and lower learning_rate and larger number of (regular) epochs 

model_list, chi2_list, loss_list = DCTR.train_loop(train_data, plt_data, model=best_model, lowest_chi2=lowest_chi2, batch_sizes=[8*8192, 16*8192], repeat=3, super_epochs=4, 
                                                   starting_super_epoch=10, train_dir = train_dir, epochs=25, learning_rate=0.0005)


if len(model_list) >= 1:
    best_model_list.extend(model_list)
    lowest_chi2_list.extend(chi2_list)

best_model = best_model_list[-1]
lowest_chi2 = lowest_chi2_list[-1]


138512
starting super_epoch 10

starting training with batch_size: 65536 and 25 epochs
starting with weights from model: ./train_20240504/super_epoch_7/run_0/s-7_b-196608_r-0.tf
starting run 0 of super_epoch 10 with batch_size 65536
loaded neural network model: ./train_20240504/super_epoch_7/run_0/s-7_b-196608_r-0.tf
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets

Epoch 11: ReduceLROnPlateau reducing learning rate to 4.6191764704417436e-05.
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
IN

In [20]:
K.clear_session() 
print(gc.collect()) # cpu gabage collection

# continue training loop with best model as starting point and small dropout

model_list, chi2_list, loss_list = DCTR.train_loop(train_data, plt_data, model=best_model, lowest_chi2=lowest_chi2, batch_sizes=[8*8192, 16*8192], repeat=6, super_epochs=4, 
                                                   starting_super_epoch=15, train_dir = train_dir, epochs=20, learning_rate=0.0002, dropout=0.1)


if len(model_list) >= 1:
    best_model_list.extend(model_list)
    lowest_chi2_list.extend(chi2_list)

best_model = best_model_list[-1]
lowest_chi2 = lowest_chi2_list[-1]


111668
starting super_epoch 15

starting training with batch_size: 65536 and 20 epochs
starting with weights from model: ./train_20240504/super_epoch_7/run_0/s-7_b-196608_r-0.tf
starting run 0 of super_epoch 15 with batch_size 65536
loaded neural network model: ./train_20240504/super_epoch_7/run_0/s-7_b-196608_r-0.tf
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_15/run_0/s-15_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_15/run_0/s-15_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_15/run_0/s-15_b-65536_r-0.tf/assets

Epoch 9: ReduceLROnPlateau reducing learning rate to 4.6655999904032795e-05.
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_15/run_0/s-15_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_15/run_0/s-15_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240504/super_epoch_15/run_0/s-15_b-65536_r-0.tf/assets
INF

In [21]:
print(f'best model: {best_model}')
print(f'with mean red chi2 of: {lowest_chi2}')


best model: ./train_20240504/super_epoch_18/run_4/s-18_b-131072_r-4.tf
with mean red chi2 of: 4.13952546329058
